In [2]:
import pandas as pd
import numpy as np

In [3]:
result_cot_general = pd.read_csv('experiment_results/gpt-turbo_multiarith_cot_general_8shot.csv', index_col=0)

result_arith_general = pd.read_csv('experiment_results/gpt-turbo_multiarith_arithcot_general_8shot.csv', index_col=0)

result_varcot_general =pd.read_csv('experiment_results/gpt-turbo_multiarith_varcot_general_8shot.csv', index_col=0)

result_cot_generalnum = pd.read_csv('experiment_results/gpt-turbo_mulriarith_cot_generalnum_8shot.csv', index_col=0)


In [4]:
def acc(results):
    return np.mean(results['answer'] == results['response_answer'])

def missing(results):
    np.sum(results['response_answer'].isnull())

In [5]:
print(acc(result_cot_general))
print(acc(result_arith_general))
print(acc(result_varcot_general))
print(acc(result_cot_generalnum))

0.9816666666666667
0.96
0.9883333333333333
0.97


## cot vs. varcot vs. arithcot

In [6]:
result_arith_general.columns = [i + '_arith' for i in result_arith_general.columns]

In [7]:
result_varcot_general.columns = [i + '_var' for i in result_varcot_general.columns]

In [8]:
cot_arithcot = pd.concat([result_cot_general, result_arith_general, result_varcot_general], axis = 1)

In [9]:
cot_arithcot.drop(columns = ['question_arith', 'answer_arith', 'question_var', 'answer_var'], inplace = True)

In [10]:
cot_arithcot['median_answer'] = -1

In [11]:
for i in range(0,600):
    a1 = cot_arithcot['response_answer'][i]
    a2 = cot_arithcot['response_answer_arith'][i]
    a3 = cot_arithcot['response_answer_var'][i]

    median_answer = np.nanmedian([a1,a2,a3])
    cot_arithcot['median_answer'][i] = median_answer

C:\Users\andyw\AppData\Local\Temp\ipykernel_33760\769570786.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cot_arithcot['median_answer'][i] = median_answer


In [12]:
np.mean(cot_arithcot['answer'] == cot_arithcot['median_answer'])

0.9933333333333333

In [33]:
changes = pd.DataFrame(cot_arithcot)

for i in range(0,600):
    if(changes['answer'][i] != changes['response_answer'][i] or
       changes['answer'][i] != changes['response_answer_arith'][i] or
       changes['answer'][i] != changes['response_answer_var'][i]):
        pass
    else:
        changes.drop(index = i, inplace=True)


In [36]:
changes = changes[["question", "answer", 'median_answer',"response_answer", "response_answer_arith","response_answer_var", "mapping_arith", "response","response_arith","response_var"]]

In [37]:
changes.to_csv("error_multiarith.csv")


## GPT 3

In [5]:
#result_arith_general_gpt3 = pd.read_csv('experiment_results/gpt3_multiarith_arithcot_general_8shot.csv', index_col=0)

result_varcot_general_gpt3 =pd.read_csv('experiment_results/gpt3_multiarith_varcot_general_8shot.csv', index_col=0)

result_cot_general_gpt3 = pd.read_csv('experiment_results/gpt3_multiarith_cot_general_8shot.csv', index_col=0)

In [6]:
print(acc(result_cot_general_gpt3))

#print(acc(result_arith_general_gpt3))

print(acc(result_varcot_general_gpt3))

0.9016666666666666
0.9166666666666666
